In [5]:
import time
import pandas as pd
import numpy as np
import random
import re
from razdel import tokenize
import pymorphy3
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, precision_recall_curve, f1_score, accuracy_score, roc_auc_score, make_scorer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import RandomizedSearchCV, ParameterSampler
import xgboost as xgb
from scipy.sparse import csr_matrix
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import gc
import os
from rapidfuzz import process, fuzz
from gensim.models import FastText
import torch
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
import pickle
import requests
import optuna

# Предсказание жанров фильма с помощью моделей машинного обучения

# Алтухов Николай

Пример описания фильма из полученного датафрейма:

Любящая, преданная и бесстрашная дочь предводителя разбойников, которого активно разыскивают солдаты, находит своего тайного друга в загадочном и опасном лесу, принадлежащем ее отцу. Этот лес полон тайн и магии, его обитатели живут по своим законам, далеко от человеческих глаз. Встреча с таинственным другом не только приносит девушке утешение и радость, но и открывает перед ней мир, полный чудес и опасностей. Вместе они исследуют лес, преодолевая трудности и раскрывая его сокровенные секреты, создавая нерушимую связь, которая меняет их жизни навсегда.

Анализ облаков слов дал понять, что при использовании стоп-слов из библиотеки NLTK, из описаний фильмов не было удалено много ненужных слов (по типу "который", "это", "тот"), поэтому я взял расширенный список стоп-слов

In [6]:
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"
response = requests.get(url)
stop_words = set(response.text.split())


In [7]:
len(stop_words)

556

In [8]:
all_films = pd.read_excel('films.xlsx')

In [5]:
all_films

,Ссылка,Название,Ориг. название,Год,Страна выпуска,Жанры,Описание,Режисер,Актеры,Рейтинг,Рейтинг IMDB,Рейтинг Кинопоиск
0,https://kinogo.uk/22795-ja-selin-dion-2024.html,Я – Селин Дион (2024),I Am: Celine Dion,Год выпуска:2024,"Страна:Канада,США","Жанр:Фильмы,Биография,Документальный",Фильм погружает зрителей в повседневную жизнь ...,Режиссер:Ирен Тейлор Бродски,"В ролях:Селин Дион, Nelson Angélil, Eddy Angél...",4.3,8.1(2490),NaN
1,https://kinogo.uk/22737-stat-karlom-lagerfeldo...,Стать Карлом Лагерфельдом (1 сезон),Becoming Karl Lagerfeld,Год выпуска:2024,Страна:Франция,"Жанр:Сериалы,Биография,Драма","Сериал, основанный на бестселлере Рафаэля Бакк...","Режиссер:Одри Эструго, Жером Салль","В ролях:Даниэль Брюль, Теодор Пеллерен, Алекс ...",4.1,7.3(1309),7.043(337)
2,https://kinogo.uk/22458-trudnye-mili-2023.html,Трудные мили (2023),Hard Miles,Год выпуска:2023,Страна:США,"Жанр:Фильмы,Биография,Драма,Спорт","Решительный социальный работник, посвятивший с...",Режиссер:Р.Дж. Дэниэл Ханна,"В ролях:Мэттью Модайн, Шон Эстин, Лесли Дэвид ...",2.8,6.8,NaN
3,https://kinogo.uk/22358-kassandro-2023.html,Кассандро (2023),Cassandro,Год выпуска:2023,"Страна:Мексика,США","Жанр:Фильмы,Биография,Драма,Спорт","История Кассандро начинается в Эль-Пасо, где м...",Режиссер:Роджер Росс Уильямс,"В ролях:Гаэль Гарсиа Берналь, Роберта Колиндре...",3.7,6.3,NaN
4,https://kinogo.uk/21215-odna-zhizn-2023.html,Одна жизнь (2023),One Life,Год выпуска:2023,Страна:Великобритания,"Жанр:Фильмы,Биография,Драма,История",Скромного и непубличного джентльмена Николаса ...,Режиссер:Джеймс Хоуз,"В ролях:Энтони Хопкинс, Хелена Бонем Картер, Д...",4.4,7.6,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21117,https://kinogo.uk/17603-baba-jaga-protiv-1979....,Баба Яга против! (1 сезон),NaN,Год выпуска:1979,Страна:СССР,"Жанр:Мультфильмы,Фэнтези","В далекой волшебной стране, где легенды оживаю...",Режиссер:Владимир Пекарь,"В ролях:Ольга Аросева, Гарри Бардин",1.0,6.7(183),6.9(24146)
21118,https://kinogo.uk/17697-poslednjaja-missija-an...,Последняя миссия ангела: Любовь (1 сезон),"Dan, hanaui sarang",Год выпуска:2019,Страна:Корея Южная,"Жанр:Сериалы,Фэнтези","В сонме небесных светил, где каждый ангел обер...",Режиссер:Ли Джон-соп,"В ролях:Щин Хе-сон, Ким Мён-су, Ли Дон-гон, Ки...",0.0,7.8(2892),7.8(17528)
21119,https://kinogo.uk/15380-arijetti-iz-strany-lil...,Ариэтти из страны лилипутов (2010),Karigurashi no Arietti,Год выпуска:2010,Страна:Япония,"Жанр:Аниме,Фэнтези","В тихом уголке мира, где жизнь текла своим чер...",Режиссер:Хиромаса Ёнэбаяси,"В ролях:Мираи Сида, Рюносукэ Камики, Тацуя Фуд...",5.0,7.6(104112),7.9(51 305)
21120,https://kinogo.uk/16169-rozhdestvenskie-tufelk...,Рождественские туфельки (2018),A Shoe Addict's Christmas,Год выпуска:2018,Страна:США,"Жанр:Фильмы,Фэнтези","Ноэль, занимающая должность менеджера по персо...",Режиссер:Майкл Робисон,"В ролях:Кендес Камерон Буре, Люк МакФарлейн, Д...",0.0,6.7(2809),6.8(8319)


In [192]:
all_films['Страна выпуска'] = all_films['Страна выпуска'].str.replace('Страна:','')
all_films['Жанры'] = all_films['Жанры'].str.replace('Жанр:','')
all_films['Актеры'] = all_films['Актеры'].str.replace('В ролях:','')
all_films = all_films.rename(columns={'Режисер':'Режиссер'})
all_films['Режиссер'] = all_films['Режиссер'].str.replace('Режиссер:','')
all_films['Год'] = all_films['Год'].str.replace('Год выпуска:','')

In [193]:
all_films

,Ссылка,Название,Ориг. название,Год,Страна выпуска,Жанры,Описание,Режиссер,Актеры,Рейтинг,Рейтинг IMDB,Рейтинг Кинопоиск
0,https://kinogo.uk/22795-ja-selin-dion-2024.html,Я – Селин Дион (2024),I Am: Celine Dion,2024,"Канада,США","Фильмы,Биография,Документальный",Фильм погружает зрителей в повседневную жизнь ...,Ирен Тейлор Бродски,"Селин Дион, Nelson Angélil, Eddy Angélil, Terr...",4.3,8.1(2490),NaN
1,https://kinogo.uk/22737-stat-karlom-lagerfeldo...,Стать Карлом Лагерфельдом (1 сезон),Becoming Karl Lagerfeld,2024,Франция,"Сериалы,Биография,Драма","Сериал, основанный на бестселлере Рафаэля Бакк...","Одри Эструго, Жером Салль","Даниэль Брюль, Теодор Пеллерен, Алекс Лутц, Ар...",4.1,7.3(1309),7.043(337)
2,https://kinogo.uk/22458-trudnye-mili-2023.html,Трудные мили (2023),Hard Miles,2023,США,"Фильмы,Биография,Драма,Спорт","Решительный социальный работник, посвятивший с...",Р.Дж. Дэниэл Ханна,"Мэттью Модайн, Шон Эстин, Лесли Дэвид Бэйкер, ...",2.8,6.8,NaN
3,https://kinogo.uk/22358-kassandro-2023.html,Кассандро (2023),Cassandro,2023,"Мексика,США","Фильмы,Биография,Драма,Спорт","История Кассандро начинается в Эль-Пасо, где м...",Роджер Росс Уильямс,"Гаэль Гарсиа Берналь, Роберта Колиндрес, Перла...",3.7,6.3,NaN
4,https://kinogo.uk/21215-odna-zhizn-2023.html,Одна жизнь (2023),One Life,2023,Великобритания,"Фильмы,Биография,Драма,История",Скромного и непубличного джентльмена Николаса ...,Джеймс Хоуз,"Энтони Хопкинс, Хелена Бонем Картер, Джонни Фл...",4.4,7.6,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21117,https://kinogo.uk/17603-baba-jaga-protiv-1979....,Баба Яга против! (1 сезон),NaN,1979,СССР,"Мультфильмы,Фэнтези","В далекой волшебной стране, где легенды оживаю...",Владимир Пекарь,"Ольга Аросева, Гарри Бардин",1.0,6.7(183),6.9(24146)
21118,https://kinogo.uk/17697-poslednjaja-missija-an...,Последняя миссия ангела: Любовь (1 сезон),"Dan, hanaui sarang",2019,Корея Южная,"Сериалы,Фэнтези","В сонме небесных светил, где каждый ангел обер...",Ли Джон-соп,"Щин Хе-сон, Ким Мён-су, Ли Дон-гон, Ким Бо-ми,...",0.0,7.8(2892),7.8(17528)
21119,https://kinogo.uk/15380-arijetti-iz-strany-lil...,Ариэтти из страны лилипутов (2010),Karigurashi no Arietti,2010,Япония,"Аниме,Фэнтези","В тихом уголке мира, где жизнь текла своим чер...",Хиромаса Ёнэбаяси,"Мираи Сида, Рюносукэ Камики, Тацуя Фудзивара, ...",5.0,7.6(104112),7.9(51 305)
21120,https://kinogo.uk/16169-rozhdestvenskie-tufelk...,Рождественские туфельки (2018),A Shoe Addict's Christmas,2018,США,"Фильмы,Фэнтези","Ноэль, занимающая должность менеджера по персо...",Майкл Робисон,"Кендес Камерон Буре, Люк МакФарлейн, Джин Смар...",0.0,6.7(2809),6.8(8319)



<p style="font-size:24px; color:#333; font-weight:400;">
    Очистка текста, удаление ненужных слов и лемматизация
</p>

In [9]:
morph = pymorphy3.MorphAnalyzer()

def clean_text(text):
    # Приведение текста к нижнему регистру
    if isinstance(text, float) or text is np.nan:
        return ""
    text = text.lower()

    # Удаление ссылок
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Замена буквы ё
    text = re.sub(r'ё', 'е', text)
    # Замена запятых на пробелы, для случаев где запятая расположена без внешних пробелов, без этого соседние слова бы сливались в одно
    text = re.sub(r',', ' ', text)
    # Удаление всех символов, кроме букв, пробелов и дефисов между буквами
    text = re.sub(r"[^a-zа-я\s\-]+|(?<!\w)-|-(?!\w)", '', text)
    
    # Токенизация и лемматизация
    tokens = [token.text for token in tokenize(text)]
    lemmas = [morph.parse(word)[0].normal_form for word in tokens]
    
    # Удаление стоп-слов после лемматизации
    lemmas = [lemma for lemma in lemmas if lemma not in stop_words]
    
    return ' '.join(lemmas)

In [195]:
# Замер времени выполнения кода
start_time = time.time()

all_films['Очищенное описание'] = all_films['Описание'].apply(clean_text)
print(f"Время выполнения функции: {time.time() - start_time} секунд")

Время выполнения функции: 505.829247713089 секунд


In [196]:
all_films

,Ссылка,Название,Ориг. название,Год,Страна выпуска,Жанры,Описание,Режиссер,Актеры,Рейтинг,Рейтинг IMDB,Рейтинг Кинопоиск,Очищенное описание
0,https://kinogo.uk/22795-ja-selin-dion-2024.html,Я – Селин Дион (2024),I Am: Celine Dion,2024,"Канада,США","Фильмы,Биография,Документальный",Фильм погружает зрителей в повседневную жизнь ...,Ирен Тейлор Бродски,"Селин Дион, Nelson Angélil, Eddy Angélil, Terr...",4.3,8.1(2490),NaN,фильм погружать зритель повседневный великий и...
1,https://kinogo.uk/22737-stat-karlom-lagerfeldo...,Стать Карлом Лагерфельдом (1 сезон),Becoming Karl Lagerfeld,2024,Франция,"Сериалы,Биография,Драма","Сериал, основанный на бестселлере Рафаэля Бакк...","Одри Эструго, Жером Салль","Даниэль Брюль, Теодор Пеллерен, Алекс Лутц, Ар...",4.1,7.3(1309),7.043(337),сериал основать бестселлер рафаэль бакка кайзе...
2,https://kinogo.uk/22458-trudnye-mili-2023.html,Трудные мили (2023),Hard Miles,2023,США,"Фильмы,Биография,Драма,Спорт","Решительный социальный работник, посвятивший с...",Р.Дж. Дэниэл Ханна,"Мэттью Модайн, Шон Эстин, Лесли Дэвид Бэйкер, ...",2.8,6.8,NaN,решительный социальный работник посвятить помо...
3,https://kinogo.uk/22358-kassandro-2023.html,Кассандро (2023),Cassandro,2023,"Мексика,США","Фильмы,Биография,Драма,Спорт","История Кассандро начинается в Эль-Пасо, где м...",Роджер Росс Уильямс,"Гаэль Гарсиа Берналь, Роберта Колиндрес, Перла...",3.7,6.3,NaN,история кассандро начинаться эль-пасо молодой ...
4,https://kinogo.uk/21215-odna-zhizn-2023.html,Одна жизнь (2023),One Life,2023,Великобритания,"Фильмы,Биография,Драма,История",Скромного и непубличного джентльмена Николаса ...,Джеймс Хоуз,"Энтони Хопкинс, Хелена Бонем Картер, Джонни Фл...",4.4,7.6,8.0,скромный непубличный джентльмен николас уинтон...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21117,https://kinogo.uk/17603-baba-jaga-protiv-1979....,Баба Яга против! (1 сезон),NaN,1979,СССР,"Мультфильмы,Фэнтези","В далекой волшебной стране, где легенды оживаю...",Владимир Пекарь,"Ольга Аросева, Гарри Бардин",1.0,6.7(183),6.9(24146),далёкий волшебный легенда оживать сказка стано...
21118,https://kinogo.uk/17697-poslednjaja-missija-an...,Последняя миссия ангела: Любовь (1 сезон),"Dan, hanaui sarang",2019,Корея Южная,"Сериалы,Фэнтези","В сонме небесных светил, где каждый ангел обер...",Ли Джон-соп,"Щин Хе-сон, Ким Мён-су, Ли Дон-гон, Ким Бо-ми,...",0.0,7.8(2892),7.8(17528),сонм небесный светить ангел оберегать чистота ...
21119,https://kinogo.uk/15380-arijetti-iz-strany-lil...,Ариэтти из страны лилипутов (2010),Karigurashi no Arietti,2010,Япония,"Аниме,Фэнтези","В тихом уголке мира, где жизнь текла своим чер...",Хиромаса Ёнэбаяси,"Мираи Сида, Рюносукэ Камики, Тацуя Фудзивара, ...",5.0,7.6(104112),7.9(51 305),тихий уголок течь черёд существовать невидимый...
21120,https://kinogo.uk/16169-rozhdestvenskie-tufelk...,Рождественские туфельки (2018),A Shoe Addict's Christmas,2018,США,"Фильмы,Фэнтези","Ноэль, занимающая должность менеджера по персо...",Майкл Робисон,"Кендес Камерон Буре, Люк МакФарлейн, Джин Смар...",0.0,6.7(2809),6.8(8319),ноэль занимать должность менеджер персонал огр...


Удаление из рейтинга скобок и всего, что находится в них. Значения рейтинга сайта умножаем на 2, так как у него 5 балльная шкала, в то время как у рейтинга IMBD и Кинопоиска - десятибалльная

In [197]:
all_films['Рейтинг IMDB'] = all_films['Рейтинг IMDB'].str.replace(r'\s*\(.*\)', '', regex=True).astype(float)
all_films['Рейтинг Кинопоиск'] = all_films['Рейтинг Кинопоиск'].str.replace(r'\s*\(.*\)', '', regex=True).astype(float)
all_films['Рейтинг'] = all_films['Рейтинг']*2
all_films['Рейтинг'] = all_films['Рейтинг'].replace(0.0, np.nan)

In [198]:
all_films['Рейтинг'] = all_films['Рейтинг']/2

Расчёт взвешенного рейтинга на основе рейтингов из различных источников. Вес источников IMDB и Кинопоиск поставим выше, чем рейтинг с исходного сайта, так как у них рейтинг более надежный, как минимум из-за количества оценок

In [199]:
all_films['Общий взвешенный рейтинг'] = (
    (all_films['Рейтинг'].fillna(0) * 0.8) +
    (all_films['Рейтинг IMDB'].fillna(0) * 2) +
    (all_films['Рейтинг Кинопоиск'].fillna(0) * 2)
) / (
    (all_films['Рейтинг'].notna()).astype(int)*0.8 +
    (all_films['Рейтинг IMDB'].notna()).astype(int)*2 +
    (all_films['Рейтинг Кинопоиск'].notna()).astype(int)*2
)

In [200]:
all_films

,Ссылка,Название,Ориг. название,Год,Страна выпуска,Жанры,Описание,Режиссер,Актеры,Рейтинг,Рейтинг IMDB,Рейтинг Кинопоиск,Очищенное описание,Общий взвешенный рейтинг
0,https://kinogo.uk/22795-ja-selin-dion-2024.html,Я – Селин Дион (2024),I Am: Celine Dion,2024,"Канада,США","Фильмы,Биография,Документальный",Фильм погружает зрителей в повседневную жизнь ...,Ирен Тейлор Бродски,"Селин Дион, Nelson Angélil, Eddy Angélil, Terr...",4.3,8.1,NaN,фильм погружать зритель повседневный великий и...,7.014286
1,https://kinogo.uk/22737-stat-karlom-lagerfeldo...,Стать Карлом Лагерфельдом (1 сезон),Becoming Karl Lagerfeld,2024,Франция,"Сериалы,Биография,Драма","Сериал, основанный на бестселлере Рафаэля Бакк...","Одри Эструго, Жером Салль","Даниэль Брюль, Теодор Пеллерен, Алекс Лутц, Ар...",4.1,7.3,7.043,сериал основать бестселлер рафаэль бакка кайзе...,6.659583
2,https://kinogo.uk/22458-trudnye-mili-2023.html,Трудные мили (2023),Hard Miles,2023,США,"Фильмы,Биография,Драма,Спорт","Решительный социальный работник, посвятивший с...",Р.Дж. Дэниэл Ханна,"Мэттью Модайн, Шон Эстин, Лесли Дэвид Бэйкер, ...",2.8,6.8,NaN,решительный социальный работник посвятить помо...,5.657143
3,https://kinogo.uk/22358-kassandro-2023.html,Кассандро (2023),Cassandro,2023,"Мексика,США","Фильмы,Биография,Драма,Спорт","История Кассандро начинается в Эль-Пасо, где м...",Роджер Росс Уильямс,"Гаэль Гарсиа Берналь, Роберта Колиндрес, Перла...",3.7,6.3,NaN,история кассандро начинаться эль-пасо молодой ...,5.557143
4,https://kinogo.uk/21215-odna-zhizn-2023.html,Одна жизнь (2023),One Life,2023,Великобритания,"Фильмы,Биография,Драма,История",Скромного и непубличного джентльмена Николаса ...,Джеймс Хоуз,"Энтони Хопкинс, Хелена Бонем Картер, Джонни Фл...",4.4,7.6,8.000,скромный непубличный джентльмен николас уинтон...,7.233333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21117,https://kinogo.uk/17603-baba-jaga-protiv-1979....,Баба Яга против! (1 сезон),NaN,1979,СССР,"Мультфильмы,Фэнтези","В далекой волшебной стране, где легенды оживаю...",Владимир Пекарь,"Ольга Аросева, Гарри Бардин",1.0,6.7,6.900,далёкий волшебный легенда оживать сказка стано...,5.833333
21118,https://kinogo.uk/17697-poslednjaja-missija-an...,Последняя миссия ангела: Любовь (1 сезон),"Dan, hanaui sarang",2019,Корея Южная,"Сериалы,Фэнтези","В сонме небесных светил, где каждый ангел обер...",Ли Джон-соп,"Щин Хе-сон, Ким Мён-су, Ли Дон-гон, Ким Бо-ми,...",NaN,7.8,7.800,сонм небесный светить ангел оберегать чистота ...,7.800000
21119,https://kinogo.uk/15380-arijetti-iz-strany-lil...,Ариэтти из страны лилипутов (2010),Karigurashi no Arietti,2010,Япония,"Аниме,Фэнтези","В тихом уголке мира, где жизнь текла своим чер...",Хиромаса Ёнэбаяси,"Мираи Сида, Рюносукэ Камики, Тацуя Фудзивара, ...",5.0,7.6,7.900,тихий уголок течь черёд существовать невидимый...,7.291667
21120,https://kinogo.uk/16169-rozhdestvenskie-tufelk...,Рождественские туфельки (2018),A Shoe Addict's Christmas,2018,США,"Фильмы,Фэнтези","Ноэль, занимающая должность менеджера по персо...",Майкл Робисон,"Кендес Камерон Буре, Люк МакФарлейн, Джин Смар...",NaN,6.7,6.800,ноэль занимать должность менеджер персонал огр...,6.750000


Обработка столбцов "Жанры", "Страна выпуска" и "Год"

In [204]:
all_films.loc[:, 'Жанры'] = all_films['Жанры'].str.replace('Фильмы,|Фильмы|Сериалы,|Сериалы','', regex=True)
all_films = all_films[~(all_films['Жанры'].str.contains('родолжительность|дж', case=False))]

In [210]:
all_films.loc[:, 'Жанры'] = all_films['Жанры'].str.split(',')

In [218]:
all_films.loc[:, 'Страна выпуска'] = all_films['Страна выпуска'].str.replace('—|Боевик|Детектив|Драма|Комедия|Криминал|Мюзикл|Приключения|Семейные|Триллер|Ужасы|Фантастика|Фэнтези','-', regex=True)
all_films.loc[:, 'Страна выпуска'] = all_films['Страна выпуска'].str.replace(r'\s*\(.*\)','', regex=True)
all_films.loc[:, 'Страна выпуска'] = all_films['Страна выпуска'].str.split(',')

In [219]:
# 2006 - среднее значение среди годов выпусков, заменим некорректные значения на него
all_films.loc[:, 'Год'] = all_films["Год"].str.replace('[^0-9]+', '2006', regex=True).astype(int)

Сохранение файла в формате pickle, который не изменит типы данных, что важно для столбцов "Жанры" и "Страна выпуска", так как они содержат в себе списки (которые форматируются при загрузке в excel или csv файл). Формат списков нам нужен будет для бинаризации

In [228]:
all_films.to_pickle('Фильмы файл с предобработкой.pkl')

In [10]:
all_films = pd.read_pickle('Фильмы файл с предобработкой.pkl')

У некоторых фильмов отсутствует описание, вставим для таких вместо описания названия фильмов и применим функцию по очистке текста

In [10]:
all_films.loc[all_films[all_films['Очищенное описание']==""].index, "Очищенное описание"] = all_films[all_films['Очищенное описание']==""]['Название'].apply(clean_text)

In [11]:
all_films.loc[all_films[all_films['Общий взвешенный рейтинг'].isna()].index,'Общий взвешенный рейтинг'] = all_films['Общий взвешенный рейтинг'].mean()

In [12]:
all_films = all_films.reset_index(drop=True)

Применим tf-idf векторизатор к словам из описания. Он присваивает каждому слову численное значение, которое соответствует tf (количество вхождений слова в тексте по отношению ко всем словам) умноженное на idf (отношение общего количество документов к количеству документов, в которые входит конкретное слово, под логарифмом)

In [13]:
start_time = time.time()
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_films['Очищенное описание'])
print(f"Время выполнения: {time.time() - start_time} секунд")

Время выполнения: 1.2431409358978271 секунд


In [108]:
#tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [109]:
#del tfidf_df

In [15]:
gc.collect()

249

<p style="font-size:24px; color:#333; font-weight:400;">
    Получилась большая разреженная матрица 20тысяч * 35тысяч, поэтому нужно уменьшить ее размерность для дальнейшего применения моделей машинного обучения
</p>
<p style="font-size:24px; color:#333; font-weight:400;">
    Вместо метода PCA (который преобразует множество коррелирующих переменных в меньшее количество некоррелирующих переменных, называемых главными компонентами; эти главные компоненты представляют собой линейные комбинации исходных переменных, которые максимизируют дисперсию данных и сохраняют как можно больше информации)
    будем применять обрезанный SVD + нормализацию L2 после этого, так как это лучше сработает для разреженной не центрированной матрицы
</p>

In [16]:
start_time = time.time()

svd = TruncatedSVD(n_components=400, random_state=29)
svd_matrix = svd.fit_transform(tfidf_matrix)

# Нормализация матрицы сжатых данных
normalizer_for_tf_idf = Normalizer()
svd_matrix_normalized = normalizer_for_tf_idf.fit_transform(svd_matrix)


#svd_df = pd.DataFrame(svd_matrix_normalized, columns=[f'SVD_компонента_{i}' for i in range(1, 2501)])


explained_variance = svd.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Создание DataFrame для анализа вклада компонент
variance_df = pd.DataFrame({
    'Компонента': range(1, len(explained_variance) + 1),
    'Объясненная дисперсия': explained_variance,
    'Суммированная объясненная дисперсия': cumulative_variance
})
print(f"Время выполнения: {time.time() - start_time} секунд")

Время выполнения: 40.7378511428833 секунд


In [17]:
variance_df

,Компонента,Объясненная дисперсия,Суммированная объясненная дисперсия
0,1,0.001568,0.001568
1,2,0.003033,0.004601
2,3,0.002832,0.007434
3,4,0.002334,0.009768
4,5,0.002150,0.011917
...,...,...,...
395,396,0.000278,0.196675
396,397,0.000278,0.196953
397,398,0.000277,0.197230
398,399,0.000276,0.197506


In [ ]:
#del svd_df

In [244]:
gc.collect()

0

In [245]:
gc.collect()

0

Так как у одного фильма может быть несколько жанров и стран выпуска, применим к этим столбцам мультибинаризацию



In [18]:
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(all_films['Жанры'])
genre_df = pd.DataFrame(genre_matrix, columns=mlb.classes_)
genre_df

,Аниме,Биография,Боевик,Вестерн,Военный,Детектив,Документальный,Драма,История,Комедия,Криминал,Мультфильмы,Мюзикл,Приключения,Семейные,Спорт,Триллер,Ужасы,Фантастика,Фэнтези
0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21104,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
21105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21106,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [19]:
mlb_country = MultiLabelBinarizer()
country_matrix = mlb_country.fit_transform(all_films['Страна выпуска'])
country_df = pd.DataFrame(country_matrix, columns=mlb_country.classes_)

In [20]:
concated_df_genres_country = pd.concat([all_films, genre_df, country_df], axis=1)

In [21]:
len(genre_df.columns.to_list()+country_df.columns.to_list())

163

In [22]:
x_column_names = ["Год","Общий взвешенный рейтинг"]+country_df.columns.to_list()
x_features = concated_df_genres_country[x_column_names]
y_targets = concated_df_genres_country[genre_df.columns.to_list()]

Соединим обрезанную tf-idf матрицу с остальными фичами



In [23]:
X_combined = np.concatenate((svd_matrix_normalized, x_features.values), axis=1)

Отмасштабируем признаки для того, чтоб влияние каждого было равномерным. Используем RobustScaler, который масштабирует признаки, используя медиану и межквартильное расстояние, которые являются более робастными, чем среднее и стандартное отклонение.



In [24]:
scaler_for_tf_idf = RobustScaler()
X_combined_scaled = scaler_for_tf_idf.fit_transform(X_combined)

Разделим датафрейм на тестовый и тренировочный


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_combined_scaled, y_targets.values, test_size=0.2, random_state=29)

Применение метода машинного обучения RandomForestClassifier (используем MultiOutputClassifier, так как у нас многометочная предсказываемая переменная)


In [285]:
start_time = time.time()
model = MultiOutputClassifier(RandomForestClassifier(random_state=29))
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

# Оценка модели
print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list())}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')

print(f"Время выполнения: {time.time() - start_time} секунд")

D:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


classification_report:
                 precision    recall  f1-score   support

         Аниме       0.00      0.00      0.00        71
     Биография       0.00      0.00      0.00       258
        Боевик       0.91      0.06      0.12       793
       Вестерн       0.00      0.00      0.00        57
       Военный       0.97      0.17      0.30       218
      Детектив       0.76      0.04      0.08       507
Документальный       1.00      0.03      0.05       143
         Драма       0.68      0.72      0.70      2146
       История       0.67      0.01      0.02       246
       Комедия       0.82      0.17      0.28      1333
      Криминал       0.88      0.12      0.21       691
   Мультфильмы       0.67      0.01      0.02       236
        Мюзикл       0.00      0.00      0.00        91
   Приключения       0.60      0.01      0.02       607
      Семейные       0.64      0.02      0.04       333
         Спорт       0.90      0.19      0.31       101
       Триллер       0.

Данный метод даже без установки нестандартных параметров показал довольно плохой результат, при долгом времени выполнения. Поэтому попробуем другие модели



<p style="font-size:24px; color:#333; font-weight:400;">
    Метрика accuracy оценивает полное совпадение. Это делает её слишком строгой для задач, где каждому примеру может соответствовать несколько меток, и даже одна неправильная метка будет считаться ошибкой
</p>
<p style="font-size:24px; color:#333; font-weight:400;">
    Поэтому в данном случае лучше использовать classification_report, которая выводит подробный отчет о качестве классификации модели машинного обучения.
</p>
<ul style="font-size:24px; color:#333; font-weight:400;">
    Эта функция вычисляет и отображает основные метрики качества классификации, такие как:
    <li>
        Precision (точность) показывает, какая доля положительных предсказаний является верной.
    </li>
    <li>
        Recall (полнота) - показывает долю верных положительных предсказаний по отношению ко всем положительным образцам в данных.
    <p style="font-size:18px; color:#333; font-weight:400;">
        * разница между точностью и полнотой в том, что в точности оказывает влияние ложно положительное предсказание, а в полноте - ложно отрицательное. Чем больше модель неверно предсказывает положительные образцы - тем хуже будет метрика точности. Чем больше неверно предсказывает отрицательные - тем хуже метрика полноты.
    </p>
    </li>
    <li>
        F1 - гармоническое среднее между precision и recall.
    </li>
    <li>
        Support показывает количество истинных случаев для каждой метки в тестовом наборе данных.
    </li>
</ul>
<hr>
<ul style="font-size:24px; color:#333; font-weight:400;">
    Так как у нас задача многоклассовой классификации, где количество классов более двух, метрики качества классификации вычисляются для каждого класса отдельно, потребуется вычисление средних значений метрик по всем классам, чтобы получить общую оценку качества классификации модели. В этом случае используются метрики, основанные на micro-, macro- и weighted-усреднениях, которые позволяют получить более полное представление о качестве классификации модели
    <li>
        Micro-Averaging (микроусреднение) - все истинно положительные, ложно положительные, истинно отрицательные и ложно отрицательные для всех классов суммируются, и затем на основе этих сумм рассчитываются метрики точности, полноты и f1.
    </li>
    <li>
        Macro-Averaging (макроусреднение) - вместо суммы здесь применяется среднее значение метрик по каждому классу (в данном случае по жанру фильма)
    </li>
    <li>
        Weighted-Averaging (взвешенное усреднение) учитывает Support (количество истинных случаев) для каждого класса при вычислении среднего значения метрик. Классы с большим количеством образцов получают больший вес.
    </li>
    <li>
         Samples-Averaging (усреднение по образцам) используется в задачах многометочной классификации (то есть как раз в нашем случае), где каждый образец может принадлежать нескольким классам. В этом случае метрики рассчитываются для каждого образца, а затем усредняются.
    </li>
</ul>

In [286]:
start_time = time.time()
model = MultiOutputClassifier(LogisticRegression(random_state=29))
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.68      0.73      0.71        71
     Биография       0.57      0.33      0.42       258
        Боевик       0.71      0.53      0.61       793
       Вестерн       0.56      0.44      0.49        57
       Военный       0.71      0.56      0.63       218
      Детектив       0.60      0.32      0.42       507
Документальный       0.69      0.59      0.64       143
         Драма       0.71      0.74      0.72      2146
       История       0.54      0.30      0.38       246
       Комедия       0.70      0.56      0.63      1333
      Криминал       0.76      0.52      0.62       691
   Мультфильмы       0.59      0.40      0.48       236
        Мюзикл       0.38      0.14      0.21        91
   Приключения       0.66      0.39      0.49       607
      Семейные       0.64      0.44      0.53       333
         Спорт       0.70      0.68      0.69       101
       Триллер       0.

<p style="font-size:24px; color:#333; font-weight:400;">
    Использование логистической регрессии значительно улучшило результаты метрик
</p>

In [287]:
start_time = time.time()
model = MultiOutputClassifier(LogisticRegression(random_state=29,penalty='elasticnet', solver='saga', l1_ratio=0.7, max_iter=2000))
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")


classification_report:
                 precision    recall  f1-score   support

         Аниме       0.71      0.77      0.74        71
     Биография       0.57      0.33      0.41       258
        Боевик       0.71      0.53      0.61       793
       Вестерн       0.56      0.42      0.48        57
       Военный       0.71      0.56      0.63       218
      Детектив       0.60      0.31      0.41       507
Документальный       0.72      0.59      0.65       143
         Драма       0.70      0.74      0.72      2146
       История       0.56      0.31      0.40       246
       Комедия       0.70      0.56      0.63      1333
      Криминал       0.76      0.52      0.62       691
   Мультфильмы       0.59      0.40      0.48       236
        Мюзикл       0.33      0.11      0.17        91
   Приключения       0.66      0.39      0.49       607
      Семейные       0.64      0.44      0.52       333
         Спорт       0.72      0.67      0.70       101
       Триллер       0.

<p style="font-size:24px; color:#333; font-weight:400;">
    Я поигрался с параметрами (тестировал разные около 10 раз) логистической модели, результатов это особо не дало. Но заняло очень много времени в сравнении с обучением со стандартными параметрами
</p>

In [288]:
start_time = time.time()
model = MultiOutputClassifier(SVC(random_state=29))
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)
print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.00      0.00      0.00        71
     Биография       0.80      0.11      0.19       258
        Боевик       0.79      0.39      0.52       793
       Вестерн       0.89      0.14      0.24        57
       Военный       0.81      0.48      0.60       218
      Детектив       0.74      0.21      0.33       507
Документальный       0.89      0.29      0.43       143
         Драма       0.71      0.75      0.73      2146
       История       0.81      0.11      0.19       246
       Комедия       0.75      0.50      0.60      1333
      Криминал       0.81      0.42      0.56       691
   Мультфильмы       0.76      0.21      0.33       236
        Мюзикл       0.00      0.00      0.00        91
   Приключения       0.77      0.27      0.40       607
      Семейные       0.70      0.26      0.37       333
         Спорт       0.82      0.62      0.71       101
       Триллер       0.

<p style="font-size:24px; color:#333; font-weight:400;">
    Модель SVC заняла много времени на выполнение при установленных стандартных параметрах. Модель довольно близка к лог. регрессии, но значительно проседает по плотности. Я попробовал поменять несколько параметров, но плодов это не принесло.
</p>

Попробуем модель XGB

In [290]:
start_time = time.time()
model = MultiOutputClassifier(xgb.XGBClassifier(random_state=29,max_depth=8,n_estimators=200,n_jobs=-3))


# Обучаем модель
model.fit(X_train, y_train)

# Предсказываем значения для тестового набора
y_pred = model.predict(X_test)
print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.82      0.59      0.69        71
     Биография       0.75      0.18      0.29       258
        Боевик       0.72      0.46      0.56       793
       Вестерн       0.90      0.16      0.27        57
       Военный       0.74      0.49      0.59       218
      Детектив       0.70      0.27      0.39       507
Документальный       0.84      0.43      0.56       143
         Драма       0.71      0.74      0.73      2146
       История       0.60      0.15      0.25       246
       Комедия       0.68      0.52      0.59      1333
      Криминал       0.78      0.46      0.58       691
   Мультфильмы       0.65      0.25      0.37       236
        Мюзикл       1.00      0.01      0.02        91
   Приключения       0.72      0.34      0.46       607
      Семейные       0.64      0.31      0.42       333
         Спорт       0.79      0.58      0.67       101
       Триллер       0.

In [291]:
start_time = time.time()
model = MultiOutputClassifier(xgb.XGBClassifier(random_state=29,objective='binary:logistic',scale_pos_weight=10, max_depth=20,n_estimators=300,n_jobs=-3))


# Обучаем модель
model.fit(X_train, y_train)

# Предсказываем значения для тестового набора
y_pred = model.predict(X_test)
print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.76      0.68      0.72        71
     Биография       0.72      0.24      0.36       258
        Боевик       0.68      0.54      0.60       793
       Вестерн       0.83      0.18      0.29        57
       Военный       0.73      0.55      0.63       218
      Детектив       0.69      0.33      0.45       507
Документальный       0.78      0.51      0.62       143
         Драма       0.66      0.82      0.73      2146
       История       0.54      0.19      0.28       246
       Комедия       0.62      0.59      0.61      1333
      Криминал       0.72      0.54      0.61       691
   Мультфильмы       0.60      0.31      0.41       236
        Мюзикл       0.33      0.01      0.02        91
   Приключения       0.64      0.42      0.51       607
      Семейные       0.64      0.43      0.52       333
         Спорт       0.77      0.59      0.67       101
       Триллер       0.

In [292]:
start_time = time.time()
model = MultiOutputClassifier(LogisticRegression(random_state=29, class_weight='balanced'))
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.60      0.83      0.69        71
     Биография       0.23      0.74      0.35       258
        Боевик       0.53      0.80      0.64       793
       Вестерн       0.28      0.51      0.36        57
       Военный       0.37      0.81      0.50       218
      Детектив       0.33      0.71      0.45       507
Документальный       0.36      0.83      0.50       143
         Драма       0.71      0.73      0.72      2146
       История       0.22      0.74      0.34       246
       Комедия       0.59      0.73      0.65      1333
      Криминал       0.50      0.80      0.61       691
   Мультфильмы       0.28      0.78      0.41       236
        Мюзикл       0.10      0.65      0.18        91
   Приключения       0.43      0.75      0.55       607
      Семейные       0.34      0.82      0.48       333
         Спорт       0.40      0.79      0.53       101
       Триллер       0.

Балансировка метрик с помощью вероятностей предсказания классов:

In [298]:
model = MultiOutputClassifier(LogisticRegression(class_weight='balanced', random_state=29))
model.fit(X_train, y_train)

# Получение вероятностей
y_probs = model.predict_proba(X_test)

# Функция для получения лучших порогов
def find_best_thresholds(y_true, y_probs):
    best_thresholds = []
    for i in range(y_true.shape[1]):
        precision, recall, thresholds = precision_recall_curve(y_true[:, i], y_probs[i][:, 1])
        f1_scores = 2 * (precision * recall) / (precision + recall)
        best_threshold = thresholds[np.argmax(f1_scores)]
        best_thresholds.append(best_threshold)
    return best_thresholds

# Получение лучших порогов для каждого класса
best_thresholds = find_best_thresholds(y_test, y_probs)

# Применение порогов к вероятностям для получения предсказаний
def apply_thresholds(y_probs, thresholds):
    y_pred = np.zeros((y_probs[0].shape[0], len(thresholds)))
    for i in range(len(thresholds)):
        y_pred[:, i] = (y_probs[i][:, 1] >= thresholds[i]).astype(int)
    return y_pred

y_pred_optimal = apply_thresholds(y_probs, best_thresholds)

# Оценка модели
print("F1-Score:", f1_score(y_test, y_pred_optimal, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred_optimal))
print("ROC AUC:", roc_auc_score(y_test, y_pred_optimal, average='macro'))

# Результаты
print("Лучшие пороги для каждого класса:", best_thresholds)

F1-Score: 0.5587875192035506
Accuracy: 0.14613927048792041
ROC AUC: 0.7575808884830824
Лучшие пороги для каждого класса: [0.9589594573915681, 0.8330898662515652, 0.5901045813181742, 0.9982427045109353, 0.9467687652057447, 0.7827206790061063, 0.9571095221890115, 0.39077760993570976, 0.8637248864159197, 0.508465373215941, 0.6811535945468825, 0.8661383973794868, 0.9785178023912162, 0.6831268280446638, 0.7784878272799656, 0.998573272812711, 0.5388745650129488, 0.8208014665243862, 0.7714253518263244, 0.9999066264687098]


C:\Users\perfe\AppData\Local\Temp\ipykernel_13864\2701049408.py:12: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)


In [300]:
print(f'classification_report:\n {classification_report(y_test, y_pred_optimal, target_names=genre_df.columns.to_list(), zero_division=0)}')

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.72      0.72      0.72        71
     Биография       0.41      0.57      0.48       258
        Боевик       0.58      0.75      0.65       793
       Вестерн       0.62      0.37      0.46        57
       Военный       0.63      0.63      0.63       218
      Детектив       0.52      0.50      0.51       507
Документальный       0.59      0.64      0.61       143
         Драма       0.66      0.84      0.74      2146
       История       0.39      0.52      0.44       246
       Комедия       0.60      0.73      0.66      1333
      Криминал       0.62      0.68      0.65       691
   Мультфильмы       0.46      0.58      0.51       236
        Мюзикл       0.25      0.27      0.26        91
   Приключения       0.54      0.62      0.58       607
      Семейные       0.50      0.68      0.58       333
         Спорт       0.79      0.65      0.71       101
       Триллер       0.

In [301]:
model = MultiOutputClassifier(LogisticRegression(random_state=29))
model.fit(X_train, y_train)

# Получение вероятностей
y_probs = model.predict_proba(X_test)

# Функция для получения лучших порогов
def find_best_thresholds(y_true, y_probs):
    best_thresholds = []
    for i in range(y_true.shape[1]):
        precision, recall, thresholds = precision_recall_curve(y_true[:, i], y_probs[i][:, 1])
        f1_scores = 2 * (precision * recall) / (precision + recall)
        best_threshold = thresholds[np.argmax(f1_scores)]
        best_thresholds.append(best_threshold)
    return best_thresholds

# Получение лучших порогов для каждого класса
best_thresholds = find_best_thresholds(y_test, y_probs)

# Применение порогов к вероятностям для получения предсказаний
def apply_thresholds(y_probs, thresholds):
    y_pred = np.zeros((y_probs[0].shape[0], len(thresholds)))
    for i in range(len(thresholds)):
        y_pred[:, i] = (y_probs[i][:, 1] >= thresholds[i]).astype(int)
    return y_pred

y_pred_optimal = apply_thresholds(y_probs, best_thresholds)

# Оценка модели
print("F1-Score:", f1_score(y_test, y_pred_optimal, average='macro'))
print("Accuracy:", accuracy_score(y_test, y_pred_optimal))
print("ROC AUC:", roc_auc_score(y_test, y_pred_optimal, average='macro'))

# Результаты
print("Лучшие пороги для каждого класса:", best_thresholds)

F1-Score: 0.5559744089248978
Accuracy: 0.15253434391283752
ROC AUC: 0.7529734936061279
Лучшие пороги для каждого класса: [0.3532548199596991, 0.3333390818361666, 0.26972674286117876, 0.25896860530195887, 0.5799673863840131, 0.3316862845969587, 0.6486193314824975, 0.4035075064909009, 0.24358376969891318, 0.3012293667573662, 0.2923432890631355, 0.24868858911033684, 0.9202840853360449, 0.2375655185663205, 0.2647678802586971, 0.6432698136856984, 0.2798570709460108, 0.27412591609248754, 0.3336284634652326, 0.9996553631730928]


C:\Users\perfe\AppData\Local\Temp\ipykernel_13864\299458294.py:12: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)
C:\Users\perfe\AppData\Local\Temp\ipykernel_13864\299458294.py:12: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)


In [302]:
print(f'classification_report:\n {classification_report(y_test, y_pred_optimal, target_names=genre_df.columns.to_list(), zero_division=0)}')

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.69      0.79      0.74        71
     Биография       0.52      0.48      0.50       258
        Боевик       0.60      0.73      0.66       793
       Вестерн       0.48      0.54      0.51        57
       Военный       0.76      0.55      0.64       218
      Детектив       0.55      0.47      0.51       507
Документальный       0.77      0.57      0.65       143
         Драма       0.66      0.84      0.74      2146
       История       0.43      0.52      0.47       246
       Комедия       0.58      0.76      0.66      1333
      Криминал       0.62      0.68      0.65       691
   Мультфильмы       0.49      0.61      0.54       236
        Мюзикл       0.00      0.00      0.00        91
   Приключения       0.53      0.65      0.58       607
      Семейные       0.54      0.66      0.59       333
         Спорт       0.73      0.67      0.70       101
       Триллер       0.

Подбор гиперпараметров с помощью optuna:

In [374]:
def objective(trial):
    # Определение гиперпараметров
    n_estimators = trial.suggest_categorical('n_estimators', [120, 300])
    max_depth = trial.suggest_categorical('max_depth', [4, 10, 16, 30])
    learning_rate = trial.suggest_categorical('learning_rate', [0.01, 0.1, 0.4])
    gamma = trial.suggest_categorical('gamma', [0, 0.2])
    colsample_bytree = trial.suggest_categorical('colsample_bytree', [0.25, 0.5, 0.75])
    scale_pos_weight = trial.suggest_categorical('scale_pos_weight', [1, 4, 10])
    subsample = trial.suggest_categorical('subsample', [0.6, 0.8, 1.0])
    reg_alpha = trial.suggest_categorical('reg_alpha', [0, 0.1, 1, 10])
    reg_lambda = trial.suggest_categorical('reg_lambda', [0, 0.1, 1, 10])
    min_child_weight = trial.suggest_categorical('min_child_weight', [2, 5, 10])
    
    # Устанавливаем параметр `tree_method`
    tree_method = trial.suggest_categorical('tree_method', ['hist', 'approx'])


    # Инициализация модели
    xgb_model = xgb.XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        gamma=gamma,
        colsample_bytree=colsample_bytree,
        scale_pos_weight=scale_pos_weight,
        subsample=subsample,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        min_child_weight=min_child_weight,
        tree_method=tree_method,
        random_state=29,
        objective='binary:logistic'
    )
    
    # Оборачивание модели в MultiOutputClassifier
    multioutput_model = MultiOutputClassifier(xgb_model, n_jobs=-3)

    # Обучение модели
    multioutput_model.fit(X_train, y_train)

    # Предсказание значений для валидационного набора
    y_pred = multioutput_model.predict(X_test)

    # Вычисление метрики качества (f1_samples)
    f1_samples = f1_score(y_test, y_pred, average='samples')

    return f1_samples

In [375]:
start_time = time.time()

# Инициализация и запуск Optuna
optuna_xgb = optuna.create_study(direction='maximize')

optuna_xgb.optimize(objective, n_trials=12)
   
# Лучшие параметры
print(f"Лучшие параметры: {optuna_xgb.best_params}")

# Лучший результат
print(f"Лучший F1 Samples: {optuna_xgb.best_value}")

# Время выполнения
print(f"Время выполнения: {time.time() - start_time} секунд")

[I 2024-07-26 04:50:58,593] A new study created in memory with name: no-name-7e0a9c0d-840f-4aa6-9486-44570445dabc
[I 2024-07-26 04:52:59,224] Trial 0 finished with value: 0.2804963419127322 and parameters: {'n_estimators': 120, 'max_depth': 16, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.25, 'scale_pos_weight': 1, 'subsample': 1.0, 'reg_alpha': 0, 'reg_lambda': 10, 'min_child_weight': 10, 'tree_method': 'hist'}. Best is trial 0 with value: 0.2804963419127322.
[I 2024-07-26 05:15:58,786] Trial 1 finished with value: 0.47989555841284876 and parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.5, 'scale_pos_weight': 1, 'subsample': 1.0, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'min_child_weight': 5, 'tree_method': 'approx'}. Best is trial 1 with value: 0.47989555841284876.
[I 2024-07-26 05:17:22,012] Trial 2 finished with value: 0.5461248643981946 and parameters: {'n_estimators': 300, 'max_depth': 16, 'learning_rate': 0.4

Лучшие параметры: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.25, 'scale_pos_weight': 10, 'subsample': 0.6, 'reg_alpha': 0.1, 'reg_lambda': 10, 'min_child_weight': 5, 'tree_method': 'hist'}
Лучший F1 Samples: 0.6096246850154952
Время выполнения: 4815.571475028992 секунд


На основе полученных лучших гиперпараметров проведем еще подбор параметров, приближенным к этим гиперпараметрам

In [386]:
def objective(trial):
    # Определение гиперпараметров
    n_estimators = trial.suggest_categorical('n_estimators', [200, 275, 350])
    max_depth = trial.suggest_categorical('max_depth', [6, 9, 12, 18])
    learning_rate = trial.suggest_categorical('learning_rate', [0.1, 0.2])
    gamma = trial.suggest_categorical('gamma', [0, 0.01, 0.1, 1])
    colsample_bytree = trial.suggest_categorical('colsample_bytree', [0.2, 0.4, 0.6])
    scale_pos_weight = trial.suggest_categorical('scale_pos_weight', [10, 25, 40, 80, 150])
    subsample = trial.suggest_categorical('subsample', [0.5, 0.7, 0.85])
    reg_alpha = trial.suggest_categorical('reg_alpha', [0.1, 1, 4])
    reg_lambda = trial.suggest_categorical('reg_lambda', [0, 0.1, 1, 10, 20])
    min_child_weight = trial.suggest_categorical('min_child_weight', [2,3,4,5])
    tree_method = trial.suggest_categorical('tree_method', ['hist', 'approx'])


    # Инициализация модели
    xgb_model = xgb.XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        gamma=gamma,
        colsample_bytree=colsample_bytree,
        scale_pos_weight=scale_pos_weight,
        subsample=subsample,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        min_child_weight=min_child_weight,
        tree_method=tree_method,
        random_state=29,
        objective='binary:logistic',
        eval_metric='logloss'
    )
    
    # Оборачивание модели в MultiOutputClassifier
    multioutput_model = MultiOutputClassifier(xgb_model, n_jobs=-3)

    # Обучение модели
    multioutput_model.fit(X_train, y_train)

    # Предсказание значений для валидационного набора
    y_pred = multioutput_model.predict(X_test)

    # Вычисление метрики качества (f1_samples)
    f1_samples = f1_score(y_test, y_pred, average='samples')

    return f1_samples

In [387]:
start_time = time.time()

# Инициализация и запуск Optuna
optuna_xgb = optuna.create_study(direction='maximize')

optuna_xgb.optimize(objective, n_trials=35)
   
# Лучшие параметры
print(f"Лучшие параметры: {optuna_xgb.best_params}")

# Лучший результат
print(f"Лучший F1 Samples: {optuna_xgb.best_value}")

# Время выполнения
print(f"Время выполнения: {time.time() - start_time} секунд")

[I 2024-07-26 07:23:21,353] A new study created in memory with name: no-name-bab3e3c6-8b96-44ad-b83a-6d1bf96eba7f
[I 2024-07-26 07:37:31,360] Trial 0 finished with value: 0.6038833002357399 and parameters: {'n_estimators': 350, 'max_depth': 9, 'learning_rate': 0.2, 'gamma': 0, 'colsample_bytree': 0.6, 'scale_pos_weight': 80, 'subsample': 0.5, 'reg_alpha': 1, 'reg_lambda': 10, 'min_child_weight': 2, 'tree_method': 'approx'}. Best is trial 0 with value: 0.6038833002357399.
[I 2024-07-26 07:40:27,180] Trial 1 finished with value: 0.6145512298638779 and parameters: {'n_estimators': 275, 'max_depth': 18, 'learning_rate': 0.2, 'gamma': 1, 'colsample_bytree': 0.4, 'scale_pos_weight': 40, 'subsample': 0.7, 'reg_alpha': 1, 'reg_lambda': 20, 'min_child_weight': 5, 'tree_method': 'hist'}. Best is trial 1 with value: 0.6145512298638779.
[I 2024-07-26 07:48:40,246] Trial 2 finished with value: 0.6226069301507483 and parameters: {'n_estimators': 350, 'max_depth': 6, 'learning_rate': 0.2, 'gamma': 1,

Лучшие параметры: {'n_estimators': 350, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.6, 'scale_pos_weight': 10, 'subsample': 0.85, 'reg_alpha': 0.1, 'reg_lambda': 10, 'min_child_weight': 5, 'tree_method': 'approx'}
Лучший F1 Samples: 0.6271256442383869
Время выполнения: 22676.485934972763 секунд


In [388]:
best_params = optuna_xgb.best_params

In [394]:
start_time = time.time()
best_model_tf_idf = MultiOutputClassifier(xgb.XGBClassifier(random_state=29,objective='binary:logistic',eval_metric='logloss',**best_params), n_jobs=-3)
best_model_tf_idf.fit(X_train, y_train)

# Предсказание с лучшей моделью
y_pred = best_model.predict(X_test)
print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = best_model_tf_idf.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

NameError: name 'best_model' is not defined

In [395]:
y_pred = best_model_tf_idf.predict(X_test)
print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = best_model_tf_idf.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.76      0.77      0.77        71
     Биография       0.61      0.29      0.39       258
        Боевик       0.64      0.69      0.66       793
       Вестерн       0.88      0.25      0.38        57
       Военный       0.71      0.56      0.63       218
      Детектив       0.57      0.40      0.47       507
Документальный       0.83      0.57      0.68       143
         Драма       0.60      0.92      0.73      2146
       История       0.56      0.28      0.37       246
       Комедия       0.55      0.76      0.64      1333
      Криминал       0.66      0.63      0.64       691
   Мультфильмы       0.62      0.43      0.51       236
        Мюзикл       0.33      0.03      0.06        91
   Приключения       0.59      0.53      0.56       607
      Семейные       0.61      0.50      0.55       333
         Спорт       0.78      0.66      0.72       101
       Триллер       0.

На данный момент это лучшая модель из полученных, сохраним ее и попробуем улучшить результат с помощью других методов векторизации

In [26]:
# Сохранение объекта tf_idf векторизатора
with open('tf_idf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

In [27]:
# Сохранение объекта SVD
with open('svd_tf_idf.pkl', 'wb') as f:
    pickle.dump(svd, f)

In [28]:
# Сохранение объекта normalizer
with open('normalizer_for_tf_idf.pkl', 'wb') as f:
    pickle.dump(normalizer_for_tf_idf, f)

In [29]:
# Сохранение объекта scaler
with open('scaler_for_tf_idf.pkl', 'wb') as f:
    pickle.dump(scaler_for_tf_idf, f)

In [396]:
with open('best_model_tf_idf.pkl', 'wb') as f:
    pickle.dump(best_model_tf_idf, f)


_____________________
___________
________________

Применим метод векторизации FastText. FastText разбивает каждое слово на n-граммы. Например, слово "привет" может быть разбито на такие как: "<пр", "при", "рив", "иве", "вет", "ет>". Начальный и конечный символы < и > используются для указания начала и конца слова.
Это позволяет модели учитывать внутреннюю структуру слов и лучше работать с редкими и неизвестными словами. Например, слова "бегать" и "бегун" будут иметь схожие n-граммы, что поможет модели понять их взаимосвязь. FastText учитывает контекст, в котором встречаются слова, улучшая их векторные представления. Это означает, что вектора слов будут различаться в зависимости от того, в каком контексте они используются.


In [30]:
start_time = time.time()
sentences = [desc.split() for desc in all_films['Очищенное описание']]

# Обучение модели FastText
fasttext_model = FastText(sentences, vector_size=250, window=9, min_count=4, epochs=20, workers=4, seed=29, alpha=0.015, negative=7)

# Получение векторного представления для каждого документа
def get_document_vector(model, document):
    words = document.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:  # Если нет слов, которые присутствуют в модели
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Применение функции ко всем описаниям
all_films['FastText vector'] = all_films['Очищенное описание'].apply(lambda x: get_document_vector(fasttext_model, x))

# Преобразование векторов в numpy array
X_fasttext = np.array(all_films['FastText vector'].tolist())
print(f"Время выполнения: {time.time() - start_time} секунд")

Время выполнения: 543.422061920166 секунд


In [31]:
with open("fasttext_model.pkl", "wb") as f:
    pickle.dump(fasttext_model, f)

<p style="font-size:24px; color:#333; font-weight:400;">
    Соединяем фичи с матрицей FastText'a и масштабируем 
</p>

In [32]:
X_combined_fasttext = np.concatenate((X_fasttext, x_features.values), axis=1)

In [33]:
scaler_for_fasttext = RobustScaler()
X_combined_fasttext_scaled = scaler_for_fasttext.fit_transform(X_combined_fasttext)

In [34]:
with open("scaler_for_fasttext.pkl", "wb") as f:
    pickle.dump(scaler_for_fasttext, f)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_combined_fasttext_scaled, y_targets.values, test_size=0.2, random_state=29)

In [412]:
start_time = time.time()
model = MultiOutputClassifier(LogisticRegression(random_state=29,max_iter=2000))
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.71      0.65      0.68        71
     Биография       0.55      0.25      0.34       258
        Боевик       0.70      0.47      0.56       793
       Вестерн       0.57      0.21      0.31        57
       Военный       0.76      0.53      0.62       218
      Детектив       0.66      0.30      0.41       507
Документальный       0.68      0.56      0.62       143
         Драма       0.70      0.74      0.72      2146
       История       0.63      0.26      0.37       246
       Комедия       0.69      0.55      0.61      1333
      Криминал       0.75      0.46      0.57       691
   Мультфильмы       0.62      0.36      0.45       236
        Мюзикл       0.41      0.10      0.16        91
   Приключения       0.69      0.34      0.45       607
      Семейные       0.64      0.38      0.48       333
         Спорт       0.73      0.61      0.67       101
       Триллер       0.


Логистическая модель получилась почти аналогичной по метрикам какие получились при tf-idf векторизаторе без подбора гиперпараметров


In [416]:

def objective_svd(trial):
    # Определение гиперпараметров
    c_value = trial.suggest_categorical('C', [0.1, 0.4, 1, 4, 10])
    class_weight = trial.suggest_categorical('class_weight', ['balanced', None])
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    max_iter = trial.suggest_categorical('max_iter', [1000, 2000, 5000])


    # Инициализация модели
    svc_model = SVC(
        max_iter=max_iter,
        C=c_value,
        class_weight=class_weight,
        kernel=kernel,
        verbose=1
    )
    
    # Оборачивание модели в MultiOutputClassifier
    multioutput_model = MultiOutputClassifier(svc_model, n_jobs=-3)

    # Обучение модели
    multioutput_model.fit(X_train, y_train)

    # Предсказание значений для валидационного набора
    y_pred = multioutput_model.predict(X_test)

    # Вычисление метрики качества (f1_samples)
    f1_samples = f1_score(y_test, y_pred, average='samples')

    return f1_samples

In [417]:
start_time = time.time()

# Инициализация и запуск Optuna
optuna_svd = optuna.create_study(direction='maximize')

optuna_svd.optimize(objective_svd, n_trials=12)
   
# Лучшие параметры
print(f"Лучшие параметры: {optuna_svd.best_params}")

# Лучший результат
print(f"Лучший F1 Samples: {optuna_svd.best_value}")

# Время выполнения
print(f"Время выполнения: {time.time() - start_time} секунд")

[I 2024-07-26 18:34:49,851] A new study created in memory with name: no-name-5a1bf20f-efab-4f90-a7d2-3722f8082046
[I 2024-07-26 18:35:15,074] Trial 0 finished with value: 0.2998324723035548 and parameters: {'C': 0.1, 'class_weight': 'balanced', 'kernel': 'linear', 'max_iter': 1000}. Best is trial 0 with value: 0.2998324723035548.
[I 2024-07-26 18:37:45,560] Trial 1 finished with value: 0.18928858348476077 and parameters: {'C': 0.1, 'class_weight': 'balanced', 'kernel': 'poly', 'max_iter': 5000}. Best is trial 0 with value: 0.2998324723035548.
[I 2024-07-26 18:38:57,536] Trial 2 finished with value: 0.17343084459781677 and parameters: {'C': 0.1, 'class_weight': 'balanced', 'kernel': 'rbf', 'max_iter': 2000}. Best is trial 0 with value: 0.2998324723035548.
[I 2024-07-26 18:40:03,143] Trial 3 finished with value: 0.3032574567437104 and parameters: {'C': 4, 'class_weight': None, 'kernel': 'linear', 'max_iter': 5000}. Best is trial 3 with value: 0.3032574567437104.
[I 2024-07-26 18:40:47,50

Лучшие параметры: {'C': 10, 'class_weight': 'balanced', 'kernel': 'poly', 'max_iter': 5000}
Лучший F1 Samples: 0.5268861106260443
Время выполнения: 990.5213692188263 секунд


In [418]:
best_params_fasttext_svd = optuna_svd.best_params

In [419]:
start_time = time.time()
best_model_fasttext_svd = MultiOutputClassifier(SVC(random_state=29,**best_params_fasttext_svd), n_jobs=-3)
best_model_fasttext_svd.fit(X_train, y_train)

# Предсказание с лучшей моделью
y_pred = best_model_fasttext_svd.predict(X_test)
print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = best_model_fasttext_svd.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.64      0.32      0.43        71
     Биография       0.46      0.37      0.41       258
        Боевик       0.50      0.61      0.55       793
       Вестерн       0.43      0.21      0.28        57
       Военный       0.57      0.50      0.53       218
      Детектив       0.39      0.45      0.42       507
Документальный       0.67      0.52      0.59       143
         Драма       0.60      0.71      0.65      2146
       История       0.38      0.31      0.34       246
       Комедия       0.49      0.71      0.58      1333
      Криминал       0.52      0.59      0.55       691
   Мультфильмы       0.48      0.46      0.47       236
        Мюзикл       0.24      0.14      0.18        91
   Приключения       0.43      0.50      0.46       607
      Семейные       0.49      0.53      0.51       333
         Спорт       0.60      0.58      0.59       101
       Триллер       0.

Налучшая из полученных SVС модель показала результат чуть хуже чем логистической регрессия без подбора гиперпараметров.
Поэтому попробуем подобрать гиперпараметры для логистической регрессии

In [423]:
gc.collect()

0

In [427]:
def objective(trial):
    # Определение гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet', None])
    solver = trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
    C = trial.suggest_float('C', 1e-4, 1e2, log=True)
    max_iter = trial.suggest_int('max_iter', 250, 1500)
    tol = trial.suggest_float('tol', 1e-5, 1e-1, log=True)
    class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])
    dual = trial.suggest_categorical('dual', [True, False]) if penalty == 'l2' else False
    
    
    # Обработка несовместимых гиперпараметров
    if penalty == 'l1' and solver not in ['liblinear', 'saga']:
        return -float('inf')  # 'l1' несовместим с 'newton-cg', 'lbfgs', 'sag'
    if penalty == 'elasticnet' and solver != 'saga':
        return -float('inf')  # 'elasticnet' совместим только с 'saga'
    if penalty is None and solver not in ['newton-cg', 'lbfgs', 'sag', 'saga']:
        return -float('inf')  # 'none' несовместим с 'liblinear'
    if solver in ['newton-cg', 'lbfgs', 'sag'] and dual:
        return -float('inf')  # 'dual' несовместим с 'newton-cg', 'lbfgs', 'sag'
    
    # Дополнительные гиперпараметры для 'saga' и 'elasticnet'
    l1_ratio = trial.suggest_uniform('l1_ratio', 0.0, 1.0) if penalty == 'elasticnet' else None

    # Определение параметров модели
    logreg_params = {
        'penalty': penalty,
        'C': C,
        'solver': solver,
        'max_iter': max_iter,
        'tol': tol,
        'l1_ratio': l1_ratio,
        'random_state': 29,
        'class_weight': class_weight,
        'dual': dual
    }
    log_model_fasttext = LogisticRegression(**logreg_params)
    
    # Оборачивание модели в MultiOutputClassifier
    multioutput_model = MultiOutputClassifier(log_model_fasttext, n_jobs=-3)

    # Обучение модели
    multioutput_model.fit(X_train, y_train)

    # Предсказание значений для валидационного набора
    y_pred = multioutput_model.predict(X_test)

    # Вычисление метрики качества (f1_samples)
    f1_samples = f1_score(y_test, y_pred, average='samples')

    return f1_samples

In [428]:
start_time = time.time()

# Инициализация и запуск Optuna
optuna_fasttext_log = optuna.create_study(direction='maximize')

optuna_fasttext_log.optimize(objective, n_trials=25)
   
# Лучшие параметры
print(f"Лучшие параметры: {optuna_fasttext_log.best_params}")

# Лучший результат
print(f"Лучший F1 Samples: {optuna_fasttext_log.best_value}")

# Время выполнения
print(f"Время выполнения: {time.time() - start_time} секунд")

[I 2024-07-26 19:46:07,763] A new study created in memory with name: no-name-1fa3b9c1-b5e9-4a3f-86ca-74651ffd5389
[I 2024-07-26 19:46:16,102] Trial 0 finished with value: 0.5335368637069113 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 3.415944384189007, 'max_iter': 907, 'tol': 0.06835801784965886, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5335368637069113.
[I 2024-07-26 19:46:29,867] Trial 1 finished with value: 0.5373418340822972 and parameters: {'penalty': None, 'solver': 'lbfgs', 'C': 0.003827433225317579, 'max_iter': 793, 'tol': 2.977308383744814e-05, 'class_weight': 'balanced'}. Best is trial 1 with value: 0.5373418340822972.
[I 2024-07-26 19:46:32,111] Trial 2 finished with value: 0.540489040145601 and parameters: {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.07453463325022439, 'max_iter': 1440, 'tol': 0.014658310472103895, 'class_weight': None}. Best is trial 2 with value: 0.540489040145601.
[I 2024-07-26 19:46:32,111] Trial 3 finished with va

Лучшие параметры: {'penalty': None, 'solver': 'sag', 'C': 0.22158496362452165, 'max_iter': 1036, 'tol': 2.8664916442546347e-05, 'class_weight': None}
Лучший F1 Samples: 0.5639949726046363
Время выполнения: 3003.653995037079 секунд


In [429]:
best_params_fasttext_log = optuna_fasttext_log.best_params

In [430]:
start_time = time.time()
best_model_fasttext_log = MultiOutputClassifier(LogisticRegression(random_state=29,**best_params_fasttext_log), n_jobs=-3)
best_model_fasttext_log.fit(X_train, y_train)

# Предсказание с лучшей моделью
y_pred = best_model_fasttext_log.predict(X_test)
print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = best_model_fasttext_svd.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.65      0.63      0.64        71
     Биография       0.55      0.25      0.34       258
        Боевик       0.70      0.47      0.56       793
       Вестерн       0.52      0.23      0.32        57
       Военный       0.74      0.54      0.62       218
      Детектив       0.65      0.29      0.40       507
Документальный       0.67      0.57      0.62       143
         Драма       0.70      0.74      0.72      2146
       История       0.59      0.24      0.35       246
       Комедия       0.69      0.55      0.61      1333
      Криминал       0.75      0.46      0.57       691
   Мультфильмы       0.60      0.36      0.45       236
        Мюзикл       0.33      0.10      0.15        91
   Приключения       0.68      0.34      0.46       607
      Семейные       0.65      0.40      0.50       333
         Спорт       0.74      0.64      0.69       101
       Триллер       0.

In [ ]:
with open('best_model_fasttext_log.pkl', 'wb') as f:
    pickle.dump(best_model_fasttext_log, f)

Попробуем еще один векторизатор. Будем использовать BERT. BERT является одной из самых передовых моделей в области обработки естественного языка, которая обеспечивает высокую точность и эффективность при обработке текста. Так же учитывает контекст.


In [38]:
# Загрузка токенизатора и модели RuBERT
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')




# Переключение на использование GPU, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Включение смешанной точности, если доступно
scaler = torch.cuda.amp.GradScaler() if device == torch.device('cuda') else None

def get_bert_embeddings(df, tokenizer, model, device, max_length=512, batch_size=16):
    model.eval()  # Переключение модели в режим оценки

    all_embeddings = []
    n = len(df)

    with torch.no_grad():  # Отключение градиентов для ускорения
        for i in range(0, n, batch_size):
            batch_texts = df['Очищенное описание'][i:i + batch_size].tolist()

            # Токенизация и преобразование в тензоры
            encoded_input = tokenizer(batch_texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt').to(device)

            # Использование смешанной точности, если доступно
            with torch.cuda.amp.autocast(enabled=scaler is not None):
                outputs = model(**encoded_input)
                # Использование эмбеддингов из последнего слоя
                embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
                all_embeddings.extend(embeddings)

    return all_embeddings

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [433]:
start_time = time.time()
embeddings = get_bert_embeddings(all_films, tokenizer, model, device, batch_size=16)
print(f"Время выполнения: {time.time() - start_time} секунд")


Время выполнения: 3279.6967582702637 секунд


In [439]:
start_time = time.time()

svd_bert = TruncatedSVD(n_components=150, random_state=29)
svd_bert_embeddings = svd_bert.fit_transform(embeddings)

# Нормализация матрицы сжатых данных
svd_embeddings_normalized = normalize(svd_bert_embeddings)


explained_variance = svd_bert.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Создание DataFrame для анализа вклада компонент
variance_df = pd.DataFrame({
    'Компонента': range(1, len(explained_variance) + 1),
    'Объясненная дисперсия': explained_variance,
    'Суммированная объясненная дисперсия': cumulative_variance
})
print(f"Время выполнения: {time.time() - start_time} секунд")

Время выполнения: 1.547325611114502 секунд


<p style="font-size:24px; color:#333; font-weight:400;">
    Сохранение обученных (fited) SVD и нормализатора для преобразования новых данных
</p>

In [440]:
# Сохранение объекта SVD
with open('svd_bert.pkl', 'wb') as f:
    pickle.dump(svd_bert, f)

In [70]:
torch.cuda.empty_cache()

In [76]:
del embeddings
gc.collect()

0

In [87]:

del sberbert_embeddings
gc.collect()

2575

In [62]:
del xlm_roberta_embeddings
gc.collect()

0

In [88]:
X_sberbert_combined = np.concatenate((svd_sberbert_embeddings_normalized, x_features.values), axis=1)
scaler_robust_for_sberbert = RobustScaler()
X_sberbert_combined_scaled = scaler_robust_for_sberbert.fit_transform(X_sberbert_combined)
X_train, X_test, y_train, y_test = train_test_split(X_sberbert_combined_scaled, y_targets.values, test_size=0.2, random_state=29)

In [66]:
start_time = time.time()
model = MultiOutputClassifier(LogisticRegression(random_state=29,max_iter=2000))
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.73      0.68      0.70        71
     Биография       0.57      0.20      0.30       258
        Боевик       0.67      0.44      0.53       793
       Вестерн       0.60      0.21      0.31        57
       Военный       0.73      0.47      0.57       218
      Детектив       0.60      0.16      0.26       507
Документальный       0.68      0.53      0.60       143
         Драма       0.69      0.73      0.71      2146
       История       0.61      0.22      0.33       246
       Комедия       0.67      0.49      0.56      1333
      Криминал       0.72      0.40      0.51       691
   Мультфильмы       0.62      0.34      0.44       236
        Мюзикл       0.35      0.08      0.13        91
   Приключения       0.67      0.29      0.41       607
      Семейные       0.65      0.33      0.44       333
         Спорт       0.78      0.61      0.69       101
       Триллер       0.

In [89]:
start_time = time.time()
model = MultiOutputClassifier(LogisticRegression(random_state=29,max_iter=2000))
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.63      0.55      0.59        71
     Биография       0.45      0.21      0.29       258
        Боевик       0.68      0.49      0.57       793
       Вестерн       0.52      0.28      0.36        57
       Военный       0.72      0.52      0.60       218
      Детектив       0.60      0.24      0.34       507
Документальный       0.66      0.48      0.56       143
         Драма       0.68      0.72      0.70      2146
       История       0.55      0.26      0.35       246
       Комедия       0.65      0.50      0.57      1333
      Криминал       0.72      0.43      0.54       691
   Мультфильмы       0.60      0.35      0.44       236
        Мюзикл       0.30      0.09      0.14        91
   Приключения       0.69      0.36      0.47       607
      Семейные       0.62      0.38      0.47       333
         Спорт       0.71      0.54      0.62       101
       Триллер       0.

In [444]:
X_bert_combined = np.concatenate((svd_embeddings_normalized, x_features.values), axis=1)

In [445]:
scaler_robust_for_bert = RobustScaler()
X_bert_combined_scaled = scaler_robust_for_bert.fit_transform(X_bert_combined)

In [446]:
X_train, X_test, y_train, y_test = train_test_split(X_bert_combined_scaled, y_targets.values, test_size=0.2, random_state=29)

In [447]:
start_time = time.time()
model = MultiOutputClassifier(LogisticRegression(random_state=29,max_iter=2000))
model.fit(X_train, y_train)

# Предсказание
y_pred = model.predict(X_test)

print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = model.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.80      0.75      0.77        71
     Биография       0.58      0.27      0.37       258
        Боевик       0.70      0.51      0.59       793
       Вестерн       0.67      0.35      0.46        57
       Военный       0.71      0.57      0.63       218
      Детектив       0.64      0.28      0.39       507
Документальный       0.72      0.59      0.65       143
         Драма       0.70      0.75      0.72      2146
       История       0.53      0.26      0.35       246
       Комедия       0.67      0.50      0.57      1333
      Криминал       0.73      0.45      0.56       691
   Мультфильмы       0.64      0.39      0.48       236
        Мюзикл       0.41      0.10      0.16        91
   Приключения       0.69      0.35      0.46       607
      Семейные       0.68      0.40      0.50       333
         Спорт       0.79      0.64      0.71       101
       Триллер       0.

Логистическая регрессия показывает аналогичные результаты как и на других векторизаторах


In [448]:
gc.collect()

551

In [450]:
def objective_bert(trial):
    # Определение гиперпараметров
    n_estimators = trial.suggest_categorical('n_estimators', [200, 275, 350])
    max_depth = trial.suggest_categorical('max_depth', [6, 9, 12, 18])
    learning_rate = trial.suggest_categorical('learning_rate', [0.01, 0.1, 0.4])
    gamma = trial.suggest_categorical('gamma', [0, 0.01, 0.1, 1])
    colsample_bytree = trial.suggest_categorical('colsample_bytree', [0.2, 0.4, 0.6])
    scale_pos_weight = trial.suggest_categorical('scale_pos_weight', [7, 10, 25, 50])
    subsample = trial.suggest_categorical('subsample', [0.5, 0.7, 0.85])
    reg_alpha = trial.suggest_categorical('reg_alpha', [0.1, 1, 4])
    reg_lambda = trial.suggest_categorical('reg_lambda', [0, 0.1, 1, 10, 20])
    min_child_weight = trial.suggest_categorical('min_child_weight', [3,5,7,10])
    tree_method = trial.suggest_categorical('tree_method', ['hist', 'approx'])


    # Инициализация модели
    xgb_model = xgb.XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        gamma=gamma,
        colsample_bytree=colsample_bytree,
        scale_pos_weight=scale_pos_weight,
        subsample=subsample,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        min_child_weight=min_child_weight,
        tree_method=tree_method,
        random_state=29,
        objective='binary:logistic',
        eval_metric='logloss'
    )
    
    # Оборачивание модели в MultiOutputClassifier
    multioutput_model = MultiOutputClassifier(xgb_model, n_jobs=-3)

    # Обучение модели
    multioutput_model.fit(X_train, y_train)

    # Предсказание значений для валидационного набора
    y_pred = multioutput_model.predict(X_test)

    # Вычисление метрики качества (f1_samples)
    f1_samples = f1_score(y_test, y_pred, average='samples')

    return f1_samples

In [451]:
start_time = time.time()

# Инициализация и запуск Optuna
optuna_bert_xgb = optuna.create_study(direction='maximize')

optuna_bert_xgb.optimize(objective_bert, n_trials=15)
   
# Лучшие параметры
print(f"Лучшие параметры: {optuna_bert_xgb.best_params}")

# Лучший результат
print(f"Лучший F1 Samples: {optuna_bert_xgb.best_value}")

# Время выполнения
print(f"Время выполнения: {time.time() - start_time} секунд")

[I 2024-07-26 22:07:00,183] A new study created in memory with name: no-name-b4ed5ed1-61a6-4f9e-9020-3328c90d9b76
[I 2024-07-26 22:12:22,226] Trial 0 finished with value: 0.5867229814766527 and parameters: {'n_estimators': 350, 'max_depth': 12, 'learning_rate': 0.01, 'gamma': 1, 'colsample_bytree': 0.6, 'scale_pos_weight': 10, 'subsample': 0.85, 'reg_alpha': 4, 'reg_lambda': 0, 'min_child_weight': 3, 'tree_method': 'hist'}. Best is trial 0 with value: 0.5867229814766527.
[I 2024-07-26 22:13:15,137] Trial 1 finished with value: 0.5813033931698072 and parameters: {'n_estimators': 350, 'max_depth': 18, 'learning_rate': 0.4, 'gamma': 1, 'colsample_bytree': 0.6, 'scale_pos_weight': 25, 'subsample': 0.85, 'reg_alpha': 0.1, 'reg_lambda': 20, 'min_child_weight': 3, 'tree_method': 'hist'}. Best is trial 0 with value: 0.5867229814766527.
[I 2024-07-26 22:13:43,913] Trial 2 finished with value: 0.5694874826210686 and parameters: {'n_estimators': 275, 'max_depth': 6, 'learning_rate': 0.1, 'gamma':

Лучшие параметры: {'n_estimators': 350, 'max_depth': 12, 'learning_rate': 0.1, 'gamma': 0.01, 'colsample_bytree': 0.6, 'scale_pos_weight': 50, 'subsample': 0.5, 'reg_alpha': 1, 'reg_lambda': 1, 'min_child_weight': 10, 'tree_method': 'approx'}
Лучший F1 Samples: 0.5961544876514085
Время выполнения: 2960.7369315624237 секунд


In [452]:
best_bert_params = optuna_bert_xgb.best_params

In [453]:
start_time = time.time()
best_model_bert = MultiOutputClassifier(xgb.XGBClassifier(random_state=29,objective='binary:logistic',eval_metric='logloss',**best_bert_params), n_jobs=-3)
best_model_bert.fit(X_train, y_train)

# Предсказание с лучшей моделью
y_pred = best_model_bert.predict(X_test)
print(f'classification_report:\n {classification_report(y_test, y_pred, target_names=genre_df.columns.to_list(), zero_division=0)}')
accuracy = best_model_bert.score(X_test, y_test)
print(f'accuracy: {accuracy}')
print(f"Время выполнения: {time.time() - start_time} секунд")

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.68      0.77      0.72        71
     Биография       0.56      0.35      0.43       258
        Боевик       0.56      0.68      0.62       793
       Вестерн       0.71      0.26      0.38        57
       Военный       0.64      0.62      0.63       218
      Детектив       0.49      0.40      0.44       507
Документальный       0.59      0.50      0.54       143
         Драма       0.59      0.92      0.72      2146
       История       0.47      0.35      0.40       246
       Комедия       0.52      0.76      0.62      1333
      Криминал       0.58      0.61      0.59       691
   Мультфильмы       0.50      0.41      0.45       236
        Мюзикл       0.36      0.09      0.14        91
   Приключения       0.54      0.54      0.54       607
      Семейные       0.56      0.50      0.53       333
         Спорт       0.72      0.60      0.66       101
       Триллер       0.

In [454]:
gc.collect()

457

<p style="font-size:24px; color:#333; font-weight:400;">
    Сохраним модель на основе BERT-векторизатора
</p>

In [455]:
with open('best_model_bert.pkl', 'wb') as f:
    pickle.dump(best_model_bert, f)


<p style="font-size:24px; color:#333; font-weight:400;">
    Я взял 3 фильма из википедии и описания оттуда. Описания скопированы из раздела "Сюжет" в википедии, этот раздел содержит значительно больше текста, чем описания с сайта, которые мы использовали для обучения моделей.
</p>

In [90]:
test_films = pd.read_excel('фильмы для предсказания.xlsx')

In [91]:
test_films

,Ссылка,Название,Ориг. название,Год,Страна выпуска,Жанры,Описание,Режиссер,Актеры,Рейтинг,Рейтинг IMDB,Рейтинг Кинопоиск
0,https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D1%...,Начало (2010),Inception,2010,"США,Великобритания","Боевик,Детектив,Драма,Триллер,Фантастика",Доминик Кобб и Артур — «извлекатели»: они зани...,Кристофер Нолан,NaN,0,8.8,8.7
1,https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D0%...,День сурка (1993),Groundhog Day,1993,США,"Драма,Комедия,Фэнтези",Фил Коннорс — метеоролог на телевизионном кана...,Харольд Рэмис,"Билл Мюррей, Энди Макдауэлл, Крис Эллиот, Стив...",0,8.0,8.1
2,https://ru.wikipedia.org/wiki/%D0%9E%D1%82%D1%...,Отступники (2006),Отступники (2006),2006,"США,Гонконг","Драма,Криминал,Триллер","Фильм начинается с предыстории. В Бостоне, шта...",Мартин Скорсезе,"Леонардо ДиКаприо, Мэтт Дэймон, Джек Николсон,...",0,8.5,8.5


<p style="font-size:24px; color:#333; font-weight:400;">
    Приведем данные к виду, который использовался обучающими данными на входе в модель
</p>

In [92]:
test_films['Год'] = test_films['Год'].astype(int) 

In [93]:
test_films.dtypes

Ссылка                object
Название              object
Ориг. название        object
Год                    int32
Страна выпуска        object
Жанры                 object
Описание              object
Режиссер              object
Актеры                object
Рейтинг                int64
Рейтинг IMDB         float64
Рейтинг Кинопоиск    float64
dtype: object

In [94]:
test_films['Очищенное описание'] = test_films['Описание'].apply(clean_text)

In [95]:
test_films['Рейтинг'] = test_films['Рейтинг'].replace(0.0, np.nan)

In [96]:
test_films['Общий взвешенный рейтинг'] = (
    (test_films['Рейтинг'].fillna(0) * 0.8) +
    (test_films['Рейтинг IMDB'].fillna(0) * 2) +
    (test_films['Рейтинг Кинопоиск'].fillna(0) * 2)
) / (
    (test_films['Рейтинг'].notna()).astype(int)*0.8 +
    (test_films['Рейтинг IMDB'].notna()).astype(int)*2 +
    (test_films['Рейтинг Кинопоиск'].notna()).astype(int)*2
)

In [97]:
test_films['Жанры'] = test_films['Жанры'].str.split(',')
test_films['Страна выпуска'] = test_films['Страна выпуска'].str.split(',')

In [98]:
gc.collect()

191

In [99]:
genre_matrix_test = mlb.transform(test_films['Жанры'])
genre_df_test = pd.DataFrame(genre_matrix_test, columns=mlb.classes_)

In [100]:
country_matrix_test = mlb_country.transform(test_films['Страна выпуска'])
country_df_test = pd.DataFrame(country_matrix_test, columns=mlb_country.classes_)

In [101]:
concated_df_genres_country_test = pd.concat([test_films, genre_df_test, country_df_test], axis=1)

In [102]:
x_column_names_test = ["Год","Общий взвешенный рейтинг"]+country_df_test.columns.to_list()

In [103]:
concated_df_genres_country_test.loc[concated_df_genres_country_test[concated_df_genres_country_test['Общий взвешенный рейтинг'].isna()].index,'Общий взвешенный рейтинг'] = concated_df_genres_country_test['Общий взвешенный рейтинг'].mean()

In [109]:
x_features_test = concated_df_genres_country_test[x_column_names_test]
y_targets_test = concated_df_genres_country_test[genre_df_test.columns.to_list()]

In [104]:
with open('svd_bert.pkl', 'rb') as f:
    svd = pickle.load(f)

with open('best_model_bert.pkl', 'rb') as f:
    best_model = pickle.load(f)

In [484]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

# Переключение на использование GPU, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Включение смешанной точности, если доступно
scaler = torch.cuda.amp.GradScaler() if device == torch.device('cuda') else None
embeddings_test = get_bert_embeddings(test_films, tokenizer, model, device, batch_size=16)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [486]:
svd_embeddings_test = svd.transform(embeddings_test)
svd_embeddings_test_normalized = normalize(svd_embeddings_test)


In [487]:
X_combined_test_bert = np.concatenate((svd_embeddings_test_normalized, x_features_test.values), axis=1)

In [488]:
X_combined_test_bert_scaled = scaler_robust_for_bert.transform(X_combined_test_bert)

In [489]:
y_pred_new = best_model.predict(X_combined_test_bert_scaled)

In [490]:
# фактические значения
y_targets_test

,Аниме,Биография,Боевик,Вестерн,Военный,Детектив,Документальный,Драма,История,Комедия,Криминал,Мультфильмы,Мюзикл,Приключения,Семейные,Спорт,Триллер,Ужасы,Фантастика,Фэнтези
0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


In [491]:
# предсказанные значения
pd.DataFrame(y_pred_new, columns=y_targets_test.columns)

,Аниме,Биография,Боевик,Вестерн,Военный,Детектив,Документальный,Драма,История,Комедия,Криминал,Мультфильмы,Мюзикл,Приключения,Семейные,Спорт,Триллер,Ужасы,Фантастика,Фэнтези
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0


In [492]:
print(f'classification_report:\n {classification_report(y_targets_test, y_pred_new, target_names=genre_df.columns.to_list(), zero_division=0)}')

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.00      0.00      0.00         0
     Биография       0.00      0.00      0.00         0
        Боевик       0.00      0.00      0.00         1
       Вестерн       0.00      0.00      0.00         0
       Военный       0.00      0.00      0.00         0
      Детектив       0.00      0.00      0.00         1
Документальный       0.00      0.00      0.00         0
         Драма       1.00      0.67      0.80         3
       История       0.00      0.00      0.00         0
       Комедия       1.00      1.00      1.00         1
      Криминал       1.00      1.00      1.00         1
   Мультфильмы       0.00      0.00      0.00         0
        Мюзикл       0.00      0.00      0.00         0
   Приключения       0.00      0.00      0.00         0
      Семейные       0.00      0.00      0.00         0
         Спорт       0.00      0.00      0.00         0
       Триллер       0.

In [105]:
with open('svd_tf_idf.pkl', 'rb') as f:
    svd = pickle.load(f)

with open('best_model_tf_idf.pkl', 'rb') as f:
    best_model = pickle.load(f)

with open('normalizer_for_tf_idf.pkl', 'rb') as f:
    normalizer_for_tf_idf = pickle.load(f)

### Сравним полученные результаты с моделью на векторизаторе tf-idf

In [107]:
tfidf_matrix_test = tfidf_vectorizer.transform(test_films['Очищенное описание'])
svd_matrix_test = svd.transform(tfidf_matrix_test)
svd_matrix_test_normalized = normalizer_for_tf_idf.transform(svd_matrix_test)

In [110]:
X_combined_test_tf_idf = np.concatenate((svd_matrix_test_normalized, x_features_test.values), axis=1)
X_combined_test_tf_idf_scaled = scaler_for_tf_idf.transform(X_combined_test_tf_idf)

In [497]:
y_pred_tf_idf = best_model.predict(X_combined_test_tf_idf_scaled)

print(f'classification_report:\n {classification_report(y_targets_test, y_pred_tf_idf, target_names=genre_df.columns.to_list(), zero_division=0)}')

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.00      0.00      0.00         0
     Биография       0.00      0.00      0.00         0
        Боевик       0.00      0.00      0.00         1
       Вестерн       0.00      0.00      0.00         0
       Военный       0.00      0.00      0.00         0
      Детектив       0.00      0.00      0.00         1
Документальный       0.00      0.00      0.00         0
         Драма       1.00      1.00      1.00         3
       История       0.00      0.00      0.00         0
       Комедия       1.00      1.00      1.00         1
      Криминал       1.00      1.00      1.00         1
   Мультфильмы       0.00      0.00      0.00         0
        Мюзикл       0.00      0.00      0.00         0
   Приключения       0.00      0.00      0.00         0
      Семейные       0.00      0.00      0.00         0
         Спорт       0.00      0.00      0.00         0
       Триллер       1.

In [498]:
# фактические значения
y_targets_test

,Аниме,Биография,Боевик,Вестерн,Военный,Детектив,Документальный,Драма,История,Комедия,Криминал,Мультфильмы,Мюзикл,Приключения,Семейные,Спорт,Триллер,Ужасы,Фантастика,Фэнтези
0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


In [499]:
# предсказанные значения
pd.DataFrame(y_pred_tf_idf, columns=y_targets_test.columns)

,Аниме,Биография,Боевик,Вестерн,Военный,Детектив,Документальный,Драма,История,Комедия,Криминал,Мультфильмы,Мюзикл,Приключения,Семейные,Спорт,Триллер,Ужасы,Фантастика,Фэнтези
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


Модель на tf-idf векторизаторе справилась намного лучше.


In [511]:
with open('best_model_fasttext_log.pkl', 'rb') as f:
    best_model_fasttext_log = pickle.load(f)
    
with open("fasttext_model.pkl", "rb") as f:
    fasttext_model = pickle.load(f)

In [513]:
gc.collect()

0

### Сравним с моделью на векторизаторе FastText

In [517]:
test_films['FastText vector'] = test_films['Очищенное описание'].apply(lambda x: get_document_vector(fasttext_model, x))
fasttext_test = np.array(test_films['FastText vector'].tolist())

In [518]:
X_combined_test_fasttext = np.concatenate((fasttext_test, x_features_test.values), axis=1)
X_combined_test_fasttext_scaled = scaler_for_fasttext.transform(X_combined_test_fasttext)

In [520]:
y_pred_fasttext = best_model_fasttext_log.predict(X_combined_test_fasttext_scaled)

print(f'classification_report:\n {classification_report(y_targets_test, y_pred_fasttext, target_names=genre_df.columns.to_list(), zero_division=0)}')

classification_report:
                 precision    recall  f1-score   support

         Аниме       0.00      0.00      0.00         0
     Биография       0.00      0.00      0.00         0
        Боевик       0.00      0.00      0.00         1
       Вестерн       0.00      0.00      0.00         0
       Военный       0.00      0.00      0.00         0
      Детектив       0.00      0.00      0.00         1
Документальный       0.00      0.00      0.00         0
         Драма       1.00      1.00      1.00         3
       История       0.00      0.00      0.00         0
       Комедия       0.00      0.00      0.00         1
      Криминал       1.00      1.00      1.00         1
   Мультфильмы       0.00      0.00      0.00         0
        Мюзикл       0.00      0.00      0.00         0
   Приключения       0.00      0.00      0.00         0
      Семейные       0.00      0.00      0.00         0
         Спорт       0.00      0.00      0.00         0
       Триллер       1.

In [521]:
y_targets_test

,Аниме,Биография,Боевик,Вестерн,Военный,Детектив,Документальный,Драма,История,Комедия,Криминал,Мультфильмы,Мюзикл,Приключения,Семейные,Спорт,Триллер,Ужасы,Фантастика,Фэнтези
0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


In [522]:
pd.DataFrame(y_pred_fasttext, columns=y_targets_test.columns)

,Аниме,Биография,Боевик,Вестерн,Военный,Детектив,Документальный,Драма,История,Комедия,Криминал,Мультфильмы,Мюзикл,Приключения,Семейные,Спорт,Триллер,Ужасы,Фантастика,Фэнтези
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


В итоге, модель на tf-idf векторизаторе справилась лучше всего. Но по метрикам все варианты оказались очень близки